In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

VANILLA_CUDA_EVENTS = "vanillacuda_api.csv"
DEV_SEGFIT_EVENTS = "devsegfit_nvtx.csv"
DEV_SEGFIT_API = "devsegfit_api.csv"
ACTIONS_CSV = "actions.csv"

In [16]:
bins = [None] * 32
malloc_sizes = []
free_sizes = []
with open(ACTIONS_CSV, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if bins[int(row[0])] != None:
            free_sizes.append(bins[int(row[0])])
        malloc_sizes.append(int(row[1]))
        bins[int(row[0])] = int(row[1])
        
for i in bins:
    free_sizes.append(i)

In [18]:
free = []
malloc = []
memorygraph = [(0, 0)]
saved = (free_sizes.copy(), malloc_sizes.copy())

with open(VANILLA_CUDA_EVENTS, 'r') as f:
  reader = csv.reader(f, delimiter='\t', skipinitialspace=True)
  next(reader)
  for row in reader:
    if row[2][-2:] == "μs":
      duration = float(row[2][:-3]) * 1000
    elif row[2][-2:] == "ms":
      duration = float(row[2][:-3])*1000000
    elif row[2][-2:] == "ns":
      duration = float(row[2][:-3])
    else:
      print(row)
      raise ValueError("Unknown time unit")
    if row[0] == "cudaFree":
      free.append([int(float(row[1][:-1])*10**9), duration])
      memoryfootprint = memorygraph[-1][1] - free_sizes.pop(0)
      memorygraph.append((int(float(row[1][:-1])*10**9), memoryfootprint))
    elif row[0] == "cudaMalloc":
      malloc.append([int(float(row[1][:-1])*10**9), duration])
      memoryfootprint = memorygraph[-1][1] + malloc_sizes.pop(0)
      memorygraph.append((int(float(row[1][:-1])*10**9), memoryfootprint))
        
vanillacuda_free_data = np.array(free, dtype=np.float32)
vanillacuda_malloc_data = np.array(malloc, dtype=np.float32)
free_sizes = saved[0]
malloc_sizes = saved[1]

In [ ]:
memorygraph_np = np.array(memorygraph, dtype=np.float32)

plt.plot(memorygraph_np[:,0]/1000000, memorygraph_np[:,1]/1024/1024, label="Memory footprint")

In [19]:
free = []
malloc = []
memorygraph = [(0, 0)]
saved = (free_sizes.copy(), malloc_sizes.copy())

with open(DEV_SEGFIT_EVENTS, 'r') as f:
  reader = csv.reader(f, delimiter='\t', skipinitialspace=True)
  next(reader)
  for row in reader:
    if row[2][-2:] == "μs":
      duration = float(row[2][:-3]) * 1000
    elif row[2][-2:] == "ms":
      duration = float(row[2][:-3])*1000000
    elif row[2][-2:] == "ns":
      duration = float(row[2][:-3])
    else:
      print(row)
      raise ValueError("Unknown time unit")
    if row[0] == "free":
      free.append([int(float(row[1][:-1])*10**9), duration])
    elif row[0] == "malloc":
      malloc.append([int(float(row[1][:-1])*10**9), duration])
      # malloc_sizes.pop(0)
    # elif row[0] == "cuMemCreate":
    #   memoryfootprint = memorygraph[-1][1] + malloc_sizes.pop(0)
    #   memorygraph.append((int(float(row[1][:-1])*10**9), memoryfootprint))

with open(DEV_SEGFIT_API, 'r') as f:
  reader = csv.reader(f, delimiter='\t', skipinitialspace=True)
  next(reader)
  i = 0
  for row in reader:
    if row[2] == "cuMemCreate":
      # Find the element in malloc that is immediately before current time and add that memory size to the memory graph
      while malloc[i,0] < int(float(row[1][:-1])*10**9):
        i += 1
      memoryfootprint = memorygraph[-1][1] + malloc_sizes[i-1,1]

devsegfit_free_data = np.array(free, dtype=np.float32)
devsegfit_malloc_data = np.array(malloc, dtype=np.float32)
free_sizes = saved[0]
malloc_sizes = saved[1]

In [ ]:
memorygraph_np = np.array(memorygraph, dtype=np.float32)

plt.plot(memorygraph_np[:,0]/1000000, memorygraph_np[:,1]/1024/1024, label="Memory footprint")